In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [173]:
rb = pd.read_csv('../input/rb/rb_loc_cleaned.csv')


In [172]:
rb.to_csv('../input/rb/rb_loc_cleaned.csv',index=False)

In [3]:
rb.dtypes

year           int64
Name          object
Team          object
Week           int64
Opponent      object
att            int64
yds            int64
td             int64
tar            int64
rec_yds        int64
def_rk         int64
h/a_1        float64
surface_0    float64
salary         int64
points       float64
dtype: object

In [175]:
rb['salary'] = rb['salary'].replace('null', 0)

In [135]:
def create_sorted_dict (df):
    df_sorted = df.sort_values(['year','Name','Week','Opponent'])
    sorted_dict = {}
    for player in df_sorted['Name']:
        sorted_dict['{}'.format(player)] = {'year':df_sorted[df_sorted['Name']==player]['year'],
                                            'points':df_sorted[df_sorted['Name']==player]['points'],
                                            'def_rk':df_sorted[df_sorted['Name']==player]['def_rk'],
                                            'surface_0':df_sorted[df_sorted['Name']==player]['surface_0'],
                                            'h/a_1':df_sorted[df_sorted['Name']== player]['h/a_1'],
                                            'salary':df_sorted[df_sorted['Name']==player]['salary']
                                           }
       
    return sorted_dict

In [54]:
len(rb_sorted_dict['Jordan Howard']['h/a_1'])

35

In [176]:
rb_sorted_dict = create_sorted_dict(rb)

In [177]:
rb_sorted_dict.keys()

dict_keys(['Adrian Peterson', 'Ahmad Bradshaw', 'Akeem Hunt', 'Alfred Blue', 'Alfred Morris', 'Alonzo Harris', 'Ameer Abdullah', 'Andre Ellington', 'Andre Williams', 'Anthony Dixon', 'Anthony Sherman', 'Antone Smith', 'Antonio Andrews', 'Arian Foster', 'Austin Johnson', 'Benjamin Cunningham', 'Bernard Pierce', 'Bilal Powell', 'Bishop Sankey', 'Bobby Rainey', 'Branden Oliver', 'Brandon Bolden', 'Bruce Miller', 'Bryce Brown', 'CJ Anderson', 'CJ Spiller', 'Cameron Artis-Payne', 'Carlos Hyde', 'Charcandrick West', 'Charles Sims', 'Chris Ivory', 'Chris Johnson', 'Chris Polk', 'Chris Thompson', 'Christine Michael', 'Cierre Wood', 'Corey Grant', 'Damien Williams', 'Dan Herron', 'Danny Woodhead', 'Darrel Young', 'Darren McFadden', 'Darren Sproles', 'David Cobb', 'David Johnson', 'DeAngelo Williams', 'DeMarco Murray', 'Denard Robinson', 'Derrick Coleman', 'Devonta Freeman', 'Dexter McCluster', 'Dion Lewis', 'Donald Brown', 'Doug Martin', 'DuJuan Harris', 'Duke Johnson', 'Eddie Lacy', 'Fitzgeral

In [12]:
rb[rb['Name']=='Jordan Howard']['points'].shape

(35,)

In [138]:
# Create a data frame to investigate the points attribute of the dataset. 


def get_points_df(df):
    total_points = df['points'].sum()
    total_games = df['points'].count()

    val_lst = []
    sal_lst = []
    games_lst = []
    wght_avg=[]
    players = []
    avg_lst=[]

    for name in list(df['Name'].unique()):
        players.append(name)
        player_games = df[df['Name'] ==name].describe().loc['count'][0]
        games_lst.append(player_games)
        avg_points = round(df[df['Name'] ==name].describe()['points'].mean(),2)
        avg_lst.append(avg_points)
        wght_avg.append((round(avg_points + (avg_points* (player_games/total_games)),2)))
        sal_avg= round(df[df['Name'] ==name].describe()['salary'].mean(),2)
        sal_lst.append(sal_avg)
        val = round(sal_avg/avg_points,2)
        val_lst.append(val)
     
    df_eng = pd.DataFrame()
    df_eng['player'] = (df['Name'].unique())
    df_eng['games'] = games_lst
    df_eng['avg_points']= avg_lst
    df_eng['wght_avg'] = wght_avg
    df_eng['sal_avg'] = sal_lst
    df_eng['val'] = val_lst
    df_eng.head()
    
    return df_eng

In [139]:
rb_points = get_points_df(rb)

In [140]:
rb_points.to_csv('../input/rb/rb_points.csv', index=False)

## Group Dataframes

In [15]:
# A function to create a dictonary of grouped data frames

def grouped_dict(csv):
    
    df= pd.read_csv(csv)
    

    yearly_dict = {}
    for year in list(df['year'].unique()):
        yearly_dict['player_{}'.format(year)] = (df[df['year']==year]).drop('year',axis=1).groupby(['Name','Week','Opponent']).sum()
        yearly_dict['player_{}'.format(year)]


    return yearly_dict

        
        

In [16]:
def conv_atts(df):
    df['salary'] = df['salary'].apply(lambda x: int(x))
    df['h/a_1'] = df['h/a_1'].astype('category')
    df['surface_0'] = df['surface_0'].astype('category')
    df['def_rk'] = df['def_rk'].astype('category')

    return df

In [178]:
rb_group = grouped_dict('../input/rb/rb_loc_cleaned.csv')
rb = conv_atts(rb)
rb.dtypes

year            int64
Name           object
Team           object
Week            int64
Opponent       object
att             int64
yds             int64
td              int64
tar             int64
rec_yds         int64
def_rk       category
h/a_1        category
surface_0    category
salary          int64
points        float64
dtype: object

In [179]:
rb_group['player_2015'].columns

Index(['att', 'yds', 'td', 'tar', 'rec_yds', 'def_rk', 'h/a_1', 'surface_0',
       'salary', 'points'],
      dtype='object')

In [143]:
def drop_col (grouped_df):
    for group in grouped_df:
        grouped_df[group].drop('def_rk', axis=1,inplace=True)
        grouped_df[group].drop('h/a_1', axis=1, inplace=True)
        grouped_df[group].drop('surface_0',axis=1, inplace=True)
        grouped_df[group].drop('salary',axis=1, inplace=True)
        grouped_df[group].drop('points',axis=1, inplace=True)
    return grouped_df

In [180]:
rb = drop_col(rb_group)
rb['player_2015'].columns

Index(['att', 'yds', 'td', 'tar', 'rec_yds'], dtype='object')

## Career Average

In [21]:
# Week 1 stats are player's career average through 2014 season
# Populate current dataframe with current year averages of players

def career_average (grouped_df1, grouped_df2=None):
    new_grouped_df1 = pd.DataFrame(index= grouped_df1.index)
    for col in grouped_df1: # Iterate through every attribute in the team_stats dataframe
        points=[]  # Create an empty list to hold the season average of the current attribute

        try:
            for player in grouped_df1.index.levels[0]:  # Iterate through every player in the team_stats df
                if player in list(grouped_df2.index.levels[0].unique()):
                    total =[grouped_df2.loc[player][col][-1]] # If player not a rookie, use final average from 2015
                else:
                    total =[]  # create an empty list to hold the weekly attributes value
                
                for week in grouped_df1.loc[player].index: # Iterate throough every week for the current player
                    total.append(grouped_df1.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                    points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list
                    
            new_grouped_df1['{}_car'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe


                                
        except:
            for player in grouped_df1.index.levels[0]:  # Iterate through every player in the team_stats df
                total = []

                for week in grouped_df1.loc[player].index: # Iterate throough every week for the current player
                    total.append(grouped_df1.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                    points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list

            new_grouped_df1['{}_car'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
    return new_grouped_df1

In [181]:
df_career_2015 = career_average(rb_group['player_2015'])
df_career_2016 = career_average(rb_group['player_2016'], rb_group['player_2015'])
df_career_2017 = career_average(rb_group['player_2017'], rb_group['player_2016'])
df_career_2018 = career_average(rb_group['player_2018'], rb_group['player_2017'])

In [182]:
player_career = pd.concat([df_career_2015, df_career_2016])
player_career = pd.concat([player_career, df_career_2017])
player_career = pd.concat([player_career, df_career_2018])

In [183]:
player_stats = pd.concat([rb_group['player_2015'],rb_group['player_2016']])
player_stats = pd.concat([player_stats,rb_group['player_2017']])
player_stats = pd.concat([player_stats,rb_group['player_2018']])
player_stats.tail()

att  yds  td  tar  rec_yds
Name              Week Opponent                            
Wayne Gallman     3    HOU         6   25   0    1        0
                  4    NO          2    6   0    3       17
Wendell Smallwood 2    TB          7   28   0    1        2
                  3    IND        10   56   1    5       35
                  4    TEN         5   39   0    5       15

In [185]:
rb_stats =  player_career

In [186]:
rb_stats.head()

att_car  yds_car  td_car  tar_car  rec_yds_car
Name            Week Opponent                                                
Adrian Peterson 1    SF          10.00     31.0    0.00     3.00        21.00
                2    DET         19.50     82.5    0.00     2.50        39.50
                3    LAC         19.67     97.0    0.67     1.67        26.33
                4    DEN         18.75     93.0    0.75     2.75        23.00
                6    KC          20.20     86.4    0.60     2.40        17.80

## Recent Player Stats
Calculate the 4 game moving average of all attributes for each player and insert resluts into positional dataframe

In [25]:
# A function designed to calcuate a player's 4 game moving average in seasons 2015,2016,2017,2018

def moving_average(grouped_df, games):
    new_grouped_df = pd.DataFrame(index = grouped_df.index)
    for col in grouped_df:  # Iterate through every attribute in the team_stats dataframe
        ma4 = []  # Create an empty list to hold the season average of the current attribute
        for player in grouped_df.index.levels[0]:  # Iterate through every player in the team_stats df
            total = [] # INstantiate a deque container with a max length of 3 to hold the three most current games for player
            count = 0 # Counter to track week number 
            for week in grouped_df.loc[player].index: # Iterate through every week for relevant player d
                if count < games: # check count value to start 3 game moving average
                    ma4.append(0)  # week 1 through 3 have 3 game movong average of 0
                    total.append(grouped_df.loc[player].loc[week][col])  # Add the value current player's attribute total from the left
                    count += 1  # Increase count
                else:  # Once 3 game moving average is avaliable
                    ma4.append(np.mean(total))  # Add the average of the 3 games held in totals 3 game moivng average list
                    total.append(grouped_df.loc[player].loc[week][col]) # Replace 3 set block with a first in first out process
        new_grouped_df['{}_ma'.format(col.lower())] = ma4  # Populate databse with 3 game moving average list
       

    return new_grouped_df   

In [187]:
player_ma_2015 = moving_average(rb_group['player_2015'],4)
player_ma_2016 = moving_average(rb_group['player_2016'],4)
player_ma_2017 = moving_average(rb_group['player_2017'],4)
player_ma_2018 = moving_average(rb_group['player_2018'],4)

In [188]:
player_yearly_ma = pd.concat([player_ma_2015, player_ma_2016])
player_yearly_ma = pd.concat([player_yearly_ma,player_ma_2017])
player_yearly_ma = pd.concat([player_yearly_ma,player_ma_2018])

In [189]:
rb_stats = pd.concat([player_yearly_ma,player_career], axis=1)
rb_stats.head()

att_ma  yds_ma  td_ma  tar_ma  rec_yds_ma  \
Name            Week Opponent                                              
Adrian Peterson 1    SF          0.00     0.0   0.00    0.00         0.0   
                2    DET         0.00     0.0   0.00    0.00         0.0   
                3    LAC         0.00     0.0   0.00    0.00         0.0   
                4    DEN         0.00     0.0   0.00    0.00         0.0   
                6    KC         18.75    93.0   0.75    2.75        23.0   

                               att_car  yds_car  td_car  tar_car  rec_yds_car  
Name            Week Opponent                                                  
Adrian Peterson 1    SF          10.00     31.0    0.00     3.00        21.00  
                2    DET         19.50     82.5    0.00     2.50        39.50  
                3    LAC         19.67     97.0    0.67     1.67        26.33  
                4    DEN         18.75     93.0    0.75     2.75        23.00  
                6    KC          20.20     86.4    0.60     2.40        17.80

## Current Stats
Weekly moving average for every year from 2015 through 2018

In [29]:
# Week 1 stats are player's career average through 2014 season
# Populate current dataframe with current year averages of players

def insert_data (grouped_df):
    new_df = pd.DataFrame(index=grouped_df.index)
    for col in grouped_df: # Iterate through every attribute in the team_stats dataframe
        points=[]  # Create an empty list to hold the season average of the current attribute

        for player in grouped_df.index.levels[0]:  # Iterate through every player in the team_stats df
            total =[]# create an empty list to hold the weekly attributes valu3

            for week in grouped_df.loc[player].index: # Iterate throough every week for the current player
                total.append(grouped_df.loc[player].loc[week][col]) # Add player's value for the current week's attribute to the total list
                points.append(round(np.mean(total),2))  # Add the average of the total list to the season average list

        new_df['{}_avg'.format(col.lower())] = points # Insert season average list for current attribute into the current stats dataframe
    return new_df

In [190]:
rb_week_2015 = insert_data(rb_group['player_2015'])
rb_week_2016 = insert_data(rb_group['player_2016'])
rb_week_2017 = insert_data(rb_group['player_2017'])
rb_week_2018 = insert_data(rb_group['player_2018'])

In [191]:
rb_week_2015.head()

att_avg  yds_avg  td_avg  tar_avg  rec_yds_avg
Name            Week Opponent                                                
Adrian Peterson 1    SF          10.00     31.0    0.00     3.00        21.00
                2    DET         19.50     82.5    0.00     2.50        39.50
                3    LAC         19.67     97.0    0.67     1.67        26.33
                4    DEN         18.75     93.0    0.75     2.75        23.00
                6    KC          20.20     86.4    0.60     2.40        17.80

In [192]:
rb_week_2015['year']=2015
rb_week_2016['year']=2016
rb_week_2017['year']=2017
rb_week_2018['year']=2018

In [193]:
rb_week = pd.concat([rb_week_2015,rb_week_2016])
rb_week = pd.concat([rb_week, rb_week_2017])
rb_week = pd.concat([rb_week, rb_week_2018])

In [194]:
year = rb_week['year']
year

Name               Week  Opponent
Adrian Peterson    1     SF          2015
                   2     DET         2015
                   3     LAC         2015
                   4     DEN         2015
                   6     KC          2015
                   7     DET         2015
                   8     CHI         2015
                   9     LAR         2015
                   10    OAK         2015
                   11    GB          2015
                   12    ATL         2015
                   13    SEA         2015
                   14    ARI         2015
                   15    CHI         2015
                   16    NYG         2015
                   17    GB          2015
Ahmad Bradshaw     6     NE          2015
                   7     NO          2015
                   8     CAR         2015
                   9     DEN         2015
                   11    ATL         2015
                   12    TB          2015
Akeem Hunt         11    NYJ         2015


In [195]:
cols = rb_week.columns.tolist()
cols = cols[-1:] +cols[:-1]
rb_week = rb_week[cols]
rb_week.head()

year  att_avg  yds_avg  td_avg  tar_avg  \
Name            Week Opponent                                            
Adrian Peterson 1    SF        2015    10.00     31.0    0.00     3.00   
                2    DET       2015    19.50     82.5    0.00     2.50   
                3    LAC       2015    19.67     97.0    0.67     1.67   
                4    DEN       2015    18.75     93.0    0.75     2.75   
                6    KC        2015    20.20     86.4    0.60     2.40   

                               rec_yds_avg  
Name            Week Opponent               
Adrian Peterson 1    SF              21.00  
                2    DET             39.50  
                3    LAC             26.33  
                4    DEN             23.00  
                6    KC              17.80

In [196]:
rb_stats = pd.concat([rb_week, rb_stats],axis=1)
rb_stats.head()

year  att_avg  yds_avg  td_avg  tar_avg  \
Name            Week Opponent                                            
Adrian Peterson 1    SF        2015    10.00     31.0    0.00     3.00   
                2    DET       2015    19.50     82.5    0.00     2.50   
                3    LAC       2015    19.67     97.0    0.67     1.67   
                4    DEN       2015    18.75     93.0    0.75     2.75   
                6    KC        2015    20.20     86.4    0.60     2.40   

                               rec_yds_avg  att_ma  yds_ma  td_ma  tar_ma  \
Name            Week Opponent                                               
Adrian Peterson 1    SF              21.00    0.00     0.0   0.00    0.00   
                2    DET             39.50    0.00     0.0   0.00    0.00   
                3    LAC             26.33    0.00     0.0   0.00    0.00   
                4    DEN             23.00    0.00     0.0   0.00    0.00   
                6    KC              17.80   18.75    93.0   0.75    2.75   

                               rec_yds_ma  att_car  yds_car  td_car  tar_car  \
Name            Week Opponent                                                  
Adrian Peterson 1    SF               0.0    10.00     31.0    0.00     3.00   
                2    DET              0.0    19.50     82.5    0.00     2.50   
                3    LAC              0.0    19.67     97.0    0.67     1.67   
                4    DEN              0.0    18.75     93.0    0.75     2.75   
                6    KC              23.0    20.20     86.4    0.60     2.40   

                               rec_yds_car  
Name            Week Opponent               
Adrian Peterson 1    SF              21.00  
                2    DET             39.50  
                3    LAC             26.33  
                4    DEN             23.00  
                6    KC              17.80

In [197]:
rb_stats.drop('year', axis=1, inplace=True)

In [198]:
rb_stats.columns

Index(['att_avg', 'yds_avg', 'td_avg', 'tar_avg', 'rec_yds_avg', 'att_ma',
       'yds_ma', 'td_ma', 'tar_ma', 'rec_yds_ma', 'att_car', 'yds_car',
       'td_car', 'tar_car', 'rec_yds_car'],
      dtype='object')

In [72]:
#A function that will covert a time series database into a supervised learning database

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [72]:
rb_stats.loc['Jordan Howard'].head()

,,att_avg,yds_avg,td_avg,tar_avg,rec_yds_avg,att_ma,yds_ma,td_ma,tar_ma,rec_yds_ma,att_car,yds_car,td_car,tar_car,rec_yds_car
Week,Opponent,,,,,,,,,,,,,,,
2,PHI,3.00,22.00,0.0,2.00,9.00,0.00,0.0,0.0,0.00,0.0,3.00,22.00,0.0,2.00,9.00
3,DAL,6.00,33.50,0.0,4.00,28.00,0.00,0.0,0.0,0.00,0.0,6.00,33.50,0.0,4.00,28.00
4,DET,11.67,59.33,0.0,4.00,25.67,0.00,0.0,0.0,0.00,0.0,11.67,59.33,0.0,4.00,25.67
5,IND,12.75,74.00,0.0,3.75,30.50,0.00,0.0,0.0,0.00,0.0,12.75,74.00,0.0,3.75,30.50
6,JAX,13.20,66.00,0.2,3.80,25.60,12.75,74.0,0.0,3.75,30.5,13.20,66.00,0.2,3.80,25.60


In [199]:
points_dict = {}
reframed_dict = {}
for player in rb_stats.index.levels[0]:
    #points_dict['{}'.format(player)]['points'] = qb_stats.loc[player]['points'][0]
    #points_dict['{}'.format(player)]['def_rk'] = qb_stats.loc[player]['def_rk']
    #points_dict['{}'.format(player)]['h/a_1'] = qb_stats.loc[player]['h/a_1']
    #points_dict['{}'.format(player)]['surface_0']= qb_stats.loc[player]['surface_0']
    #points_dict['{}'.format(player)]['salary']=qb_stats.loc[player]['salary']
    #rop_list = ['points','def_rk','h/a_1','surface_0','salary']
    #qb_stats.drop(drop_list, axis=1, inplace=True)
    reframed_dict['{}_reframed'.format(player)]= series_to_supervised(rb_stats.loc[player])

In [200]:
# A function to insert the non reframed columns back into the stats dataframe 

def insert_non_reframed(df, col_dict):
    non_reframed = ['year','def_rk','h/a_1','surface_0','salary','points']
    for player in df.index.levels[0]:
        for col in non_reframed:
            reframed_dict['{}_reframed'.format(player)][col] = col_dict['{}'.format(player)][col][1:].values

In [201]:
insert_non_reframed(rb_stats, rb_sorted_dict)

In [165]:
non_reframed = ['year','def_rk','h/a_1','surface_0','salary','points']

for player in rb_stats.index.levels[0]:
    for col in non_reframed:
        if len(reframed_dict['{}_reframed'.format(player)]['var1(t-1)']) != len(rb_sorted_dict['{}'.format(player)][col][1:].values):
            print(player)
        

Dalvin Cook
Dalvin Cook
Dalvin Cook
Dalvin Cook
Dalvin Cook
Dalvin Cook


In [95]:
reframed_dict['Todd Gurley_reframed']['var1(t)'].shape

(47,)

In [94]:
rb_sorted_dict['Todd Gurley']['points']

234      2.4
306     21.1
379     18.9
511     35.3
573     26.6
631     19.9
690     17.9
3804    11.6
822      4.0
904      4.1
981     30.7
1060    15.9
1140    16.0
1282     5.2
1360     8.0
1431    21.0
3877    13.2
1546    18.8
1613    13.7
1678    15.2
1785    11.4
1836     8.2
1891    13.6
1951    12.9
2018     6.7
2075    17.3
2144     4.6
2209    17.7
2278    11.7
2346    20.6
2419    27.6
2489    40.9
2565    37.5
2644     6.0
2705    16.0
2772    28.4
2889    24.4
2951    19.6
3016    14.6
3078    16.8
3146    21.8
3212    28.5
3278    48.0
3345    55.6
3480    26.7
3556    32.3
3626    28.6
3689    25.6
4223    25.6
Name: points, dtype: float64

In [118]:
rb[rb['Name']=='Todd Gurley']

,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,salary,points
234,2015,Todd Gurley,LAR,3,PIT,6,9,0,1,5,2,0.0,1.0,5100,2.4
306,2015,Todd Gurley,LAR,4,ARI,19,146,0,2,15,8,0.0,1.0,4400,21.1
379,2015,Todd Gurley,LAR,5,GB,30,159,0,0,0,13,0.0,1.0,4500,18.9
511,2015,Todd Gurley,LAR,7,CLE,19,128,2,5,35,16,0.0,1.0,4200,35.3
573,2015,Todd Gurley,LAR,8,SF,20,133,1,3,13,32,0.0,1.0,3000,26.6
631,2015,Todd Gurley,LAR,9,MIN,24,89,1,5,20,9,0.0,0.0,3000,19.9
690,2015,Todd Gurley,LAR,10,CHI,12,45,1,3,44,22,0.0,1.0,5900,17.9
822,2015,Todd Gurley,LAR,12,CIN,9,19,0,2,11,15,0.0,0.0,3300,4.0
904,2015,Todd Gurley,LAR,13,ARI,9,41,0,0,0,8,0.0,1.0,3000,4.1
981,2015,Todd Gurley,LAR,14,DET,16,140,2,2,7,17,0.0,0.0,3000,30.7


In [167]:
rb.columns

AttributeError: 'dict' object has no attribute 'columns'

In [202]:
rb_stats = pd.concat(reframed_dict)
rb_stats.columns

Index(['var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)',
       'var6(t-1)', 'var7(t-1)', 'var8(t-1)', 'var9(t-1)', 'var10(t-1)',
       'var11(t-1)', 'var12(t-1)', 'var13(t-1)', 'var14(t-1)', 'var15(t-1)',
       'var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)', 'var6(t)',
       'var7(t)', 'var8(t)', 'var9(t)', 'var10(t)', 'var11(t)', 'var12(t)',
       'var13(t)', 'var14(t)', 'var15(t)', 'year', 'def_rk', 'h/a_1',
       'surface_0', 'salary', 'points'],
      dtype='object')

In [207]:
cols = rb_stats.columns.tolist()
cols = cols[-6:-5]  + cols[:-6] + cols[-5:]
rb_stats = rb_stats[cols]

In [208]:
rb_stats.head()

year  var1(t-1)  var2(t-1)  var3(t-1)  \
                     Week Opponent                                          
Aaron Jones_reframed 5    DAL       2017       13.0      49.00       1.00   
                     6    MIN       2017       16.0      87.00       1.00   
                     7    NO        2017       15.0      71.67       0.67   
                     9    DET       2017       15.5      86.50       0.75   
                     10   CHI       2017       13.4      71.60       0.60   

                                    var4(t-1)  var5(t-1)  var6(t-1)  \
                     Week Opponent                                    
Aaron Jones_reframed 5    DAL            0.00       0.00        0.0   
                     6    MIN            0.50       4.50        0.0   
                     7    NO             1.67       3.33        0.0   
                     9    DET            2.50       4.25        0.0   
                     10   CHI            3.00       3.20       15.5   

                                    var7(t-1)  var8(t-1)  var9(t-1)   ...    \
                     Week Opponent                                    ...     
Aaron Jones_reframed 5    DAL             0.0       0.00        0.0   ...     
                     6    MIN             0.0       0.00        0.0   ...     
                     7    NO              0.0       0.00        0.0   ...     
                     9    DET             0.0       0.00        0.0   ...     
                     10   CHI            86.5       0.75        2.5   ...     

                                    var11(t)  var12(t)  var13(t)  var14(t)  \
                     Week Opponent                                           
Aaron Jones_reframed 5    DAL          16.00     87.00      1.00      0.50   
                     6    MIN          15.00     71.67      0.67      1.67   
                     7    NO           15.50     86.50      0.75      2.50   
                     9    DET          13.40     71.60      0.60      3.00   
                     10   CHI          11.67     61.67      0.50      2.67   

                                    var15(t)  def_rk  h/a_1  surface_0  \
                     Week Opponent                                       
Aaron Jones_reframed 5    DAL           4.50       8    0.0        0.0   
                     6    MIN           3.33       1    0.0        0.0   
                     7    NO            4.25      17    1.0        1.0   
                     9    DET           3.20      31    1.0        1.0   
                     10   CHI           2.67      11    0.0        1.0   

                                    salary  points  
                     Week Opponent                  
Aaron Jones_reframed 5    DAL         3500    23.4  
                     6    MIN         4000     5.2  
                     7    NO          5000    25.8  
                     9    DET         3000     3.1  
                     10   CHI         3000     2.2  

[5 rows x 36 columns]

In [174]:
rb[rb['Name']=='Dalvin Cook'].sort_values(['year','Week'])

,year,Name,Team,Week,Opponent,att,yds,td,tar,rec_yds,def_rk,h/a_1,surface_0,salary,points
2368,2017,Dalvin Cook,MIN,1,NO,22,127,0,5,10,17,1.0,0.0,3500,19.7
2443,2017,Dalvin Cook,MIN,2,PIT,12,64,0,3,0,18,0.0,1.0,6800,8.4
2516,2017,Dalvin Cook,MIN,3,TB,27,97,1,5,72,29,1.0,0.0,4500,27.9
2590,2017,Dalvin Cook,MIN,4,DET,13,66,1,3,8,31,1.0,0.0,4000,13.4
3507,2018,Dalvin Cook,MIN,1,SF,16,40,0,7,55,26,1.0,0.0,3000,14.5
3582,2018,Dalvin Cook,MIN,2,GB,10,38,0,5,52,9,0.0,1.0,4300,12.0
3709,2018,Dalvin Cook,MIN,4,LAR,10,20,0,0,0,11,0.0,1.0,3600,2.0


In [209]:
rb_stats.to_csv('../input/rb/rb_stats.csv', index=True)